In [1]:
!pip install transformers==4.46.2 datasets==3.1.0 torch==2.5.1+cu121 sklearn-pandas==2.2.0 numpy==1.26.4

  Using cached transformers-4.46.2-py3-none-any.whl.metadata (44 kB)
ERROR: Could not find a version that satisfies the requirement torch==2.5.1+cu121 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.5.1+cu121


In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
import pandas as pd
import datasets
import torch
import os
from fvcore.nn import FlopCountAnalysis


2024-12-04 19:35:57.382168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733340957.392961  334611 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733340957.395756  334611 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 19:35:57.406880: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


The WANDB_API_KEY is nessesary to use the Trainer class from HuggingFace and is what's used in the example notebook. However, I think we may all have to put in our own api key secret into google colab for it to work.

In [ ]:
# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [2]:
langs = ['java', 'python', 'pharo']

# langs = ['java'] # Using Java as the only language for testing purposes.
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = datasets.load_dataset('NLBSE/nlbse25-code-comment-classification')

In [3]:
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [4]:
ds['java_test'][0]

{'index': 5,
 'class': 'AbstractContractGetFileStatusTest.java',
 'comment_sentence': 'accept everything.',
 'partition': 1,
 'combo': 'accept everything. | AbstractContractGetFileStatusTest.java',
 'labels': [0, 0, 1, 0, 0, 0, 0]}

In [5]:
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

The next few functions are to preprocess the trainng and validation sets, and to let the Trainer class to evaluate how well the trainng is going after each epoch.

In [6]:
# To tokenize the text in the 'combo' column of the training dataset of each language.

def tokenize_dataset(examples):
    return tokenizer(examples['combo'], truncation=True, padding="max_length", max_length=128)

In [7]:
# Tokenizes the text 'combo' column of the dataset, changes the values of the
# labels column to float instead of int, and sets to format of the lables column
# to torch tesnsors, which is required by the Trainer class.

def preprocess_dataset(input_dataset):
  processed_dataset = input_dataset.map(tokenize_dataset, batched=True, load_from_cache_file=False)
  processed_dataset = processed_dataset.cast_column("labels", datasets.features.Sequence(datasets.features.Value("float32")))
  processed_dataset.set_format('pt')
  return processed_dataset

Evaluating the predictions.

Below is non-functional code I still haven't finnished. We need to implement a way for the logits returned from the model to be converted into binary (0 - 1) predictions for the labels, and for those labels to be compaired to the ground truth labels of the testing datasets for each language. The competition also wants us to complete this step using profiling that they have set up, and which is partially shown below. The link to the reference notebook for the competition is [here](https://colab.research.google.com/drive/1RULzasjO_nrqiXLrGze-PznFlHKtGQ4s?usp=sharing). Also, another notebook that is fairly close to what I believe we are trying to do can be found [here](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb).

In [ ]:
# scores must be global so that it can later be referenced for printing the results
scores = pd.DataFrame(columns=['lan', 'cat', 'precision', 'recall', 'f1'])

def compute_metrics(eval_pred, lang, categories):
    global scores
    scores.drop(scores[scores['lan'] == lang].index, inplace=True)

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'lan': lang,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame(metrics_list)
    scores = pd.concat([scores, temp_scores], ignore_index=True)

    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

In [53]:
def measure_runtime_and_flops(trainer, validation_dataset):
    total_time = 0
    total_flops = 0

    for batch in validation_dataset:
        inputs = {key: val.unsqueeze(0).to(trainer.model.device) for key, val in batch.items() if key in ['input_ids', 'attention_mask']}
        with torch.profiler.profile(with_flops=True, activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]) as prof:
            start_time = time.time()
            _ = trainer.model(**inputs)
            end_time = time.time()

        total_time += (end_time - start_time)
        total_flops += sum(k.flops for k in prof.key_averages()) / 1e9 

    avg_runtime = total_time / len(validation_dataset)
    avg_flops = (total_flops / len(validation_dataset)) / 1e9  # Convert to GFLOPs
    return avg_runtime, avg_flops, total_flops, total_time

In [ ]:
# Wrapper is needed because model can only take eval_pred for compute_metrics, but lang and labels are necessary for output
def compute_metrics_wrapper(eval_pred):
    return compute_metrics(eval_pred, lang, labels[lang])

In [55]:
total_flops = 0
total_time = 0

# Main Training Loop
for lang in langs:
    num_labels = len(labels[lang])
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    
    dataset = preprocess_dataset(ds[f'{lang}_train'])
    train_val_split = dataset.train_test_split(test_size=0.2, seed=27)
    train_dataset = train_val_split['train']
    validation_dataset = train_val_split['test']

    training_args = TrainingArguments(
        output_dir=f'./results_{lang}',
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir=f'./logs_{lang}',
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        learning_rate=5e-5,
        logging_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_wrapper,
    )

    trainer.train()
    metrics = trainer.evaluate()

    avg_runtime, avg_flops, total_flops, total_time = measure_runtime_and_flops(trainer, validation_dataset)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7614 [00:00<?, ? examples/s]

/tmp/ipykernel_334611/3904585414.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.106100,0.098827,0.658129,0.668911,0.649673
2,0.042200,0.084482,0.792778,0.886259,0.748799
3,0.013900,0.077687,0.851650,0.885532,0.824483


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.369900,0.344438,0.430224,0.519443,0.376532
2,0.248000,0.282514,0.557926,0.679337,0.482004
3,0.195600,0.284786,0.568065,0.647297,0.522345


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1298 [00:00<?, ? examples/s]

/tmp/ipykernel_334611/3904585414.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,F1,Precision,Recall
1,0.259800,0.313199,0.117806,0.112987,0.123055
2,0.233500,0.247622,0.338978,0.626606,0.274411
3,0.157900,0.240241,0.456346,0.571366,0.392828


In [56]:
scores

,lan,cat,precision,recall,f1
0,java,summary,0.943262,0.945946,0.944602
1,java,Ownership,0.964912,1.000000,0.982143
2,java,Expand,0.688889,0.607843,0.645833
3,java,usage,0.976019,0.931350,0.953162
4,java,Pointer,0.977143,0.904762,0.939560
5,java,deprecation,0.964286,0.900000,0.931034
6,java,rational,0.684211,0.481481,0.565217
7,python,Usage,0.986486,0.603306,0.748718
8,python,Parameters,0.829060,0.866071,0.847162
9,python,DevelopmentNotes,0.000000,0.000000,0.000000


In [57]:
max_avg_runtime = 5
max_avg_flops = 5000

# s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

avg_f1 = scores.f1.mean()

round(score(avg_f1, avg_runtime, avg_flops), 2)

0.78